# Analyzing Orange Telecoms Customer Churn Dataset

Predicting which customer is likely to cancel subscription to a service. It has become a common practise across banks, ISPs, insurance firms and credit card companies.

### Run `Apache pyspark` from the terminal:
Load `pyspark` with the [Spark-CSV](http://spark-packages.org/package/databricks/spark-csv) package.

> IPYTHON_OPTS="notebook" ~/path_to_pyspark --packages com.databricks:spark-csv_2.11:1.4.0

In [1]:
import pandas as pd
import plotly.plotly as py
import cufflinks as cf

## Fetching and Importing Churn Dataset
For this tutorial, we'll be using the Orange Telecoms Churn Dataset. It consists of cleaned customer activity data (features), along with a churn label specifying whether the customer canceled their subscription or not. The data can be fetched from BigML's S3 bucket, [churn-80](https://bml-data.s3.amazonaws.com/churn-bigml-80.csv) and [churn-20](https://bml-data.s3.amazonaws.com/churn-bigml-20.csv). The two sets are from the same batch, but have been split by an 80/20 ratio. We'll use the larger set for training and cross-validation purposes, and the smaller set for final testing and model performance evaluation. The two data sets have been included in this repository for convenience.

In [3]:
# Import dataset with spark CSV package
orange_sprk_df = sqlContext.read.load("../_datasets_downloads/churn-bigml-80.csv",
                                format='com.databricks.spark.csv',
                                header='true',
                                inferschema='true')

orange_final_dataset = sqlContext.read.load("../_datasets_downloads/churn-bigml-20.csv",
                                format='com.databricks.spark.csv',
                                header='true',
                                inferschema='true')

# Print Dataframe Schema. That's DataFrame = Dataset[Row]
orange_sprk_df.cache()
orange_sprk_df.printSchema()

root
 |-- State: string (nullable = true)
 |-- Account length: integer (nullable = true)
 |-- Area code: integer (nullable = true)
 |-- International plan: string (nullable = true)
 |-- Voice mail plan: string (nullable = true)
 |-- Number vmail messages: integer (nullable = true)
 |-- Total day minutes: double (nullable = true)
 |-- Total day calls: integer (nullable = true)
 |-- Total day charge: double (nullable = true)
 |-- Total eve minutes: double (nullable = true)
 |-- Total eve calls: integer (nullable = true)
 |-- Total eve charge: double (nullable = true)
 |-- Total night minutes: double (nullable = true)
 |-- Total night calls: integer (nullable = true)
 |-- Total night charge: double (nullable = true)
 |-- Total intl minutes: double (nullable = true)
 |-- Total intl calls: integer (nullable = true)
 |-- Total intl charge: double (nullable = true)
 |-- Customer service calls: integer (nullable = true)
 |-- Churn: boolean (nullable = true)



In [33]:
# Display first Row or Spark Dataset
print("Number of customer data:",orange_sprk_df.count())
orange_sprk_df.first()

('Number of customer data:', 2666)


Row(State=u'KS', Account length=128, Area code=415, International plan=u'No', Voice mail plan=u'Yes', Number vmail messages=25, Total day minutes=265.1, Total day calls=110, Total day charge=45.07, Total eve minutes=197.4, Total eve calls=99, Total eve charge=16.78, Total night minutes=244.7, Total night calls=91, Total night charge=11.01, Total intl minutes=10.0, Total intl calls=3, Total intl charge=2.7, Customer service calls=1, Churn=False)

In [25]:
orange_telecom_pd = orange_sprk_df.toPandas()
o
orange_telecom_pd.head(10)

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
5,AL,118,510,Yes,No,0,223.4,98,37.98,220.6,101,18.75,203.9,118,9.18,6.3,6,1.70,0,False
6,MA,121,510,No,Yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,False
7,MO,147,415,Yes,No,0,157.0,79,26.69,103.1,94,8.76,211.8,96,9.53,7.1,6,1.92,0,False
8,WV,141,415,Yes,Yes,37,258.6,84,43.96,222.0,111,18.87,326.4,97,14.69,11.2,5,3.02,0,False
9,RI,74,415,No,No,0,187.7,127,31.91,163.4,148,13.89,196.0,94,8.82,9.1,5,2.46,0,False


In [24]:
ov_stats_1 = orange_telecom_pd[['Account length', 'Total day minutes', 'Total day calls', 'Total night minutes']]

ov_stats_1.iplot(kind='box', orientation='v')



In [ ]:
ov_stats = orange_telecom_pd[['Account length', 'Customer service calls']]

